#  Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from datetime import*
import seaborn as sns
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import fbeta_score, make_scorer
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from geopy.geocoders import Nominatim
from tqdm import tqdm
from sklearn.compose import ColumnTransformer
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder
geolocator = Nominatim(user_agent="geoapiExercises")
%matplotlib inline

# Loading Datasets


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
sales = pd.read_csv('/content/drive/Shareddrives/Capstone/CAPSTONE MODEL/datosArticuloLocalizacionS.csv', sep = ';')
items = pd.read_csv('/content/drive/Shareddrives/Capstone/CAPSTONE MODEL/datosArticuloS.csv', sep = ',')
place = pd.read_csv('/content/drive/Shareddrives/Capstone/CAPSTONE MODEL/datosLocalizacionS.csv', sep = ',')
age_avg = pd.read_csv('age_avg.csv', sep = ';').dropna()
pop_dens = pd.read_csv('pop_density.csv', sep = ';').dropna()
corruption_index = pd.read_csv('corruption_index.csv', sep = ';').dropna()
avg_child_country = pd.read_csv('avg_children_country.csv', sep = ';').dropna()
city_pop = pd.read_csv('city_population.csv', sep = ';').dropna()
inflation = pd.read_csv('InflationData.csv', sep = ';')
temperature = pd.read_csv('temperature.csv', ';', encoding = 'latin-1')
unemployment = pd.read_csv('unemployment.csv', sep = ';')
fem_features_country =  pd.read_csv('fem_features_country.csv', sep=';')

# Combining Datasets

In [ ]:
sales['FECHA_PRIMERA_VENTA'] = pd.to_datetime(sales['FECHA_PRIMERA_VENTA'], format='%d/%m/%Y')  

## cities containes age avg per city, and city population
city_pop['female_ratio'] = pd.Series([i.replace(',', '.') for i in city_pop['female_ratio']]).astype(float)
cities = age_avg.merge(pop_dens, how='inner', on='city').merge(city_pop, on='city', how='left')

## unemployment
unemployment['unemployment_rate']=[str(i).replace(',', '.') for i in unemployment['unemployment_rate']]
unemployment['unemployment_rate'] =unemployment['unemployment_rate'].astype(float)
unemployment['unemployment_rate'] = unemployment['unemployment_rate'].fillna(unemployment['unemployment_rate'].mean())
cities = cities.merge(unemployment, how='left', on='city')

#temperature
temperature['FECHA_PRIMERA_VENTA'] = pd.to_datetime(temperature['FECHA_PRIMERA_VENTA'], format='%d/%m/%Y')  
temperature['tavg']= temperature['tavg'].astype(float)
temperature['tavg']= temperature['tavg'].astype(float)
temperature['tavg']= temperature['tavg'].astype(float)

#all together
cities = age_avg.merge(pop_dens, how='inner', on='city').merge(city_pop, on='city', how='left').merge(unemployment, how='left', on='city')
country_data = corruption_index.merge(avg_child_country, on='MERCADO', how='left').merge(fem_features_country, on = 'MERCADO', how='left')
sales = sales.merge(place, how = 'left', on = 'COD_LOCALIZACION').merge(items,how = 'left' , on='COD_ARTICULO').dropna()


In [ ]:
cities['population']=cities['population'].fillna(cities['population'].mean())
cities['female_ratio']=cities['female_ratio'].fillna(1.11)
cities['city'] =cities['city'].astype('str')
cities['population'] =cities['population'].replace('.','').astype('int')

# DATA TYPES

In [ ]:
sales.info()

In [ ]:
cities.info()

In [ ]:
country_data.info()

# FEATURE ENGINEERING

## SEASONAL FEATURES

### Adding year, month, week, and day

In [ ]:
sales['year'] = pd.DatetimeIndex(sales['FECHA_PRIMERA_VENTA']).year
sales['month'] = pd.DatetimeIndex(sales['FECHA_PRIMERA_VENTA']).month
sales['week'] = pd.DatetimeIndex(sales['FECHA_PRIMERA_VENTA']).week
sales['day'] = pd.DatetimeIndex(sales['FECHA_PRIMERA_VENTA']).day

## Creating end_date column
end_date = []
# create a datetime object for a specific date and time
for i, j in zip(sales['FECHA_PRIMERA_VENTA'],sales['DIAS_HASTA_SALDO']):
    # add 3 days to the datetime object
    end_date.append(i + timedelta(days=int(j)))
sales['end_date'] = end_date 

## Adding day of weeks to calculate weekends
date_object = []
date = sales['FECHA_PRIMERA_VENTA'].astype(str)
for i in date:
    date_object.append(datetime.strptime(i, "%Y-%m-%d"))
    
# Get the day of the week (Monday=0, Tuesday=1, ..., Sunday=6)
day_of_week= []
for i in date_object:
    day_of_week.append(i.isoweekday())

sales['day_of_week'] = pd.Series(day_of_week)
sales.dropna(inplace=True)
sales['weekend']=[1 if i in [5,6] else 0 for i in list(sales['day_of_week'])]

sales['end_month'] = pd.DatetimeIndex(sales['end_date']).month

### Adding days per season per campaign


In [ ]:
seasons_df = pd.read_csv('seasons_df.csv', sep = ';')
seasons_df['date'] = pd.to_datetime(seasons_df['date'], format='%d/%m/%Y')  


In [ ]:
winter_temp = []
for i,j in tqdm(zip(sales['FECHA_PRIMERA_VENTA'],sales['end_date'])):
    #start_date = df_original['start_date']
    #end_date = df_original['end_date']

  # Select the rows between the start and end dates
    mask = (seasons_df['date'] >= i) & (seasons_df['date'] <= j)
    #mask = mask.reset_index()
    #print(mask)
   
    selected_rows = seasons_df.loc[mask]
    #print(selected_rows)
    total = selected_rows['winter'].sum()
    #print(mean)
    winter_temp.append(total)
    
sales['winter_days'] = pd.Series(winter_temp)
sales['winter_days'] =sales['winter_days'].fillna(0)
sales.info()

In [ ]:
spring_temp = []
for i,j in tqdm(zip(sales['FECHA_PRIMERA_VENTA'],sales['end_date'])):
    #start_date = df_original['start_date']
    #end_date = df_original['end_date']

  # Select the rows between the start and end dates
    mask = (seasons_df['date'] >= i) & (seasons_df['date'] <= j)
    #mask = mask.reset_index()
    #print(mask)
   
    selected_rows = seasons_df.loc[mask]
    #print(selected_rows)
    total = selected_rows['spring'].sum()
    #print(mean)
    spring_temp.append(total)
    
sales['spring_days'] = spring_temp
sales.info()

In [ ]:
summer_temp = []
for i,j in tqdm(zip(sales['FECHA_PRIMERA_VENTA'],sales['end_date'])):
    #start_date = df_original['start_date']
    #end_date = df_original['end_date']

  # Select the rows between the start and end dates
    mask = (seasons_df['date'] >= i) & (seasons_df['date'] <= j)
    #mask = mask.reset_index()
    #print(mask)
   
    selected_rows = seasons_df.loc[mask]
    #print(selected_rows)
    total = selected_rows['summer'].sum()
    #print(mean)
    summer_temp.append(total)
sales['summer_days'] = summer_temp
sales.info()

In [ ]:
autumn_temp = []
for i,j in tqdm(zip(sales['FECHA_PRIMERA_VENTA'],sales['end_date'])):
    #start_date = df_original['start_date']
    #end_date = df_original['end_date']

  # Select the rows between the start and end dates
    mask = (seasons_df['date'] >= i) & (seasons_df['date'] <= j)
    #mask = mask.reset_index()
    #print(mask)
   
    selected_rows = seasons_df.loc[mask]
    #print(selected_rows)
    total = selected_rows['autumn'].sum()
    #print(mean)
    autumn_temp.append(total)
    
sales['autumn_days'] = autumn_temp
sales.info()

### Months for Each Campaign

In [ ]:
sales = sales.reset_index().drop('index', axis=1)

In [ ]:
sales_c = sales

In [ ]:
## function to add 1 or 0 to each of the month columns based on which campaigns occur during which months
jan = []
feb = []
mar = []
apr = []
may = []
jun = []
jul = []
aug = []
sep = []
octo = []
nov = []
dec = []

for i,j in zip(sales['month'], sales['end_month']):
    if i<=j:
        l = list(range(i,j+1))
    else:
        l = list(range(i,12+1))+list(range(1,j+1))
    if 1 in l:
        jan.append(1)
    else:
        jan.append(0)
    if 2 in l:
        feb.append(1)
    else:
        feb.append(0)
    if 3 in l:
        mar.append(1)
    else:
        mar.append(0) 
    if 4 in l:
        apr.append(1)
    else:
        apr.append(0)
    if 5 in l:
        may.append(1)
    else:
        may.append(0)
    if 6 in l:
        jun.append(1)
    else:
        jun.append(0)
    if 7 in l:
        jul.append(1)
    else:
        jul.append(0)
    if 8 in l:
        aug.append(1)
    else:
        aug.append(0)
    if 9 in l:
        sep.append(1)
    else:
        sep.append(0)
    if 10 in l:
        octo.append(1)
    else:
        octo.append(0)
    if 11 in l:
        nov.append(1)
    else:
        nov.append(0)
    if 12 in l:
        dec.append(1)
    else:
        dec.append(0)
        
sales['month1'] = pd.Series(jan)
sales['month2'] = pd.Series(feb)
sales['month3'] = pd.Series(mar)
sales['month4'] = pd.Series(apr)
sales['month5'] = pd.Series(may)
sales['month6'] = pd.Series(jun)
sales['month7'] = pd.Series(jul)
sales['month8'] = pd.Series(aug)
sales['month9'] = pd.Series(sep)
sales['month10'] = pd.Series(octo)
sales['month11'] = pd.Series(nov)
sales['month12'] = pd.Series(dec)


### Couting Number of Weekends per Campaign

In [ ]:
#count number of weekends based on end and start_date
def count_weekends(start_date, end_date):
    weekend_count = 0
    days = (end_date-start_date).days+1
    for i in range(days):
        day = start_date + timedelta(i)
        if day.weekday() == 5 or day.weekday() == 6:
            weekend_count+=1

    return weekend_count
weekend_count=[]


for i,j in zip(sales['FECHA_PRIMERA_VENTA'], sales['end_date']):
    weekend_count.append(count_weekends(i,j))

sales['weekend_days_count'] = weekend_count


### Count number of Major Holidays per Campaign

In [ ]:
week_end = pd.DatetimeIndex(sales['end_date']).week
sales['week_end'] = week_end
#function to count the number of holidays for each campaign(each row)
def count_holidays(start_week, end_week):
    holiday_count = 0
    #for week_s, week_end in zip(start_date, end_date):
    if (start_week <= 49 and end_week>=49) or (start_week <= 52 and end_week >= 52) :
        holiday_count+= 1
    if (start_week <= 6 and end_week>=6)  or (start_week <= 7 and  end_week >= 52):
        holiday_count += 1
    if (start_week <= 25 and end_week>=25) or (start_week <= 26 and  end_week >= 26):
        holiday_count+= 1
    if(start_week <= 46 and end_week>=46) or (start_week <= 47 and  end_week >= 47):
        holiday_count+= 1
    if (start_week <= 17 and end_week>=17) or (start_week <= 18 and  end_week >= 18):
        holiday_count += 1

    return holiday_count


holiday_n=[]
for i, j in zip(sales['week'],sales['week_end']):
    holiday_n.append(count_holidays(i,j))
    

sales['holiday_count'] = pd.Series(holiday_n)
sales['holiday_count'] = sales['holiday_count'].fillna(0)


## INCORPORATING COUNTRY STATISTICS

In [ ]:
sales = sales.merge(country_data, on=['MERCADO','year'], how='left')

In [ ]:
sales.info()

## Incorporating Average Temperature per Campaign

In [ ]:
#rounding latitude and longitude in sales dataset
sales['LATITUD']=round(sales['LATITUD'],4)
sales['LONGITUD']=round(sales['LONGITUD'],4)
temperature['LONGITUD']=temperature['LONGITUD'].astype(float)

# merging temperature to sales dataset
sales = sales.merge(temperature, on =['FECHA_PRIMERA_VENTA','LATITUD','LONGITUD'], how='left').fillna(temperature['tavg'].mean())

temperature_end =temperature
temperature_end.columns =['end_date', 'tavg', 'LATITUD', 'LONGITUD']
sales = sales.merge(temperature_end, on =['end_date','LATITUD','LONGITUD'], how='left').fillna(temperature['tavg'].mean())

#calculating average temperature for the period of the campaign
sales['tavg'] = (sales['tavg_x']+sales['tavg_y'])*0.5
sales.drop(['tavg_x', 'tavg_y'], axis=1, inplace=True)

## Average month to month US, EU, and Japan Inflation Rates

In [ ]:
sales['end_month'] = pd.DatetimeIndex(sales['end_date']).month
sales = sales.merge(inflation, on =['month','year'], how='left')

inflation_end =inflation
inflation_end.columns =['end_month', 'year','Euro_inflation','US_inflation','Japan_Inflation']
sales = sales.merge(inflation_end, on =['end_month','year'], how='left')

#calculating average inflation month/month for EU, US, and Japan
sales['Euro_inflation_x'].fillna(sales['Euro_inflation_x'].mean(), inplace=True)
sales['US_inflation_x'].fillna(sales['US_inflation_x'].mean(), inplace=True)
sales['Japan_Inflation_x'].fillna(sales['Japan_Inflation_x'].mean(), inplace=True)

sales['Euro_inflation_y'].fillna(sales['Euro_inflation_y'].mean(), inplace=True)
sales['US_inflation_y'].fillna(sales['US_inflation_y'].mean(), inplace=True)
sales['Japan_Inflation_y'].fillna(sales['Japan_Inflation_y'].mean(), inplace=True)

sales['Euro_inflation']=(sales['Euro_inflation_x']+sales['Euro_inflation_y'])*0.5
sales['US_inflation']=(sales['US_inflation_x']+sales['US_inflation_y'])*0.5
sales['Japan_inflation'] = (sales['Japan_Inflation_x']+sales['Japan_Inflation_y'])*0.5
sales.drop(['Japan_Inflation_x', 'Japan_Inflation_y', 'US_inflation_x','US_inflation_y','Euro_inflation_x','Euro_inflation_y'], axis=1, inplace=True)

## Incorporating Cities Based on Latitude and Longitude

In [ ]:
x= sales[['COD_LOCALIZACION','LATITUD','LONGITUD']]#.drop_duplicates().reset_index().drop('index', axis=1)
x=x.drop_duplicates().reset_index()
x= x[['COD_LOCALIZACION','LATITUD','LONGITUD']]

#calling function geolocator to return city and county based on latitude and longitude
city = []
lat = x['LATITUD'].astype(str)
lon = x['LONGITUD'].astype(str)
for i,j in tqdm(zip(lat,lon)):
    #print(i,j)
    a = geolocator.reverse(i+','+j).raw['address']
    if a.get('city')!= None:
        city.append(a.get('city'))
        next
    elif a.get('state_district')!= None:
        city.append(a.get('state_district'))
        next
    elif a.get('town')!=None:
        city.append(a.get('town'))
        next
    else:
        city.append(a.get('county'))
        
##Correcting typos
city[65] = 'Valencia'
city[58] = 'Valencia'
city[63] = 'Valencia'
city = pd.Series(city)

## Adding cities, to the entire dataframe
x = pd.concat([x,city], axis=1)
x.columns = ['COD_LOCALIZACION','LATITUD','LONGITUD', 'city']
x.drop(['LATITUD','LONGITUD'], axis=1, inplace=True)
sales = sales.merge(x, on='COD_LOCALIZACION', how='left')
sales = sales.merge(cities, on='city', how='left').dropna()

## Binning Categorical Variables

In [ ]:
sales.info()

### COD_TIPO

In [ ]:
a = sales['COD_TIPO'].value_counts().sum() ##17, 16, 1, 19, 10, 12, 9, 16, 8, 23,  minority<5% classes
minority = [17, 16,1,19,10,12,9,16,8,23,11]

In [ ]:
sales['COD_TIPO']= ['minority' if i in minority  else i for i in list(sales['COD_TIPO'])]

In [ ]:
sales['COD_TIPO'].value_counts()

In [ ]:
sales['COD_TIPO'].value_counts()/sales['COD_TIPO'].value_counts().sum()

### COD COMPRADOR

In [ ]:
sales['COD_COMPRADOR'].hist()

In [ ]:
sales['COD_COMPRADOR'].value_counts()

In [ ]:
sales['COD_COMPRADOR'].value_counts()/sales['COD_COMPRADOR'].value_counts().sum()

### COD SUBTIPO

In [ ]:
sales['COD_SUBTIPO']

In [ ]:
sales['COD_SUBTIPO'].value_counts()/sales['COD_SUBTIPO'].value_counts().sum()

### COD_TIPO_LOCALIZACION 

In [ ]:
sales['COD_TIPO_LOCALIZACION'].value_counts()

In [ ]:
## converting minority classes T4, T7 into a single class called T4-7
sales['COD_TIPO_LOCALIZACION'] = ['T4-7' if i in ['T4', 'T7']  else i for i in list(sales['COD_TIPO_LOCALIZACION'])]

In [ ]:
sales['COD_TIPO_LOCALIZACION'].hist()

In [ ]:
sales['COD_TIPO_LOCALIZACION'].value_counts()

## Incorporating Economic/Financial Variables

In [ ]:
import pandas_datareader as pdr
import datetime
import yfinance as yfin
yfin.pdr_override()

#### Avg Inditex Stock Price per Campaign

In [ ]:
start ='2018-01-12'
end = '2022-11-17'

In [ ]:
## calculating average Inditex price fot the period of the campaign
avg_price_stock = []
inditex = yfin.download('ITX.MC', start=start, end=end)['Adj Close']
for i, j in zip(sales['FECHA_PRIMERA_VENTA'], sales['end_date']):
    avg_price_stock.append(inditex[(inditex.index>=i)&(inditex.index<=j)].mean())
sales['avg_stock_price'] = avg_price_stock

#### Avg IBEX35 Stock Price per Campaign

In [ ]:
## calculating average IBEX35 price fot the period of the campaign

avg_ibex_stock = []
inditex = yfin.download('^IBEX', start=start, end=end)['Adj Close']
for i, j in zip(sales['FECHA_PRIMERA_VENTA'], sales['end_date']):
    avg_ibex_stock.append(inditex[(inditex.index>=i)&(inditex.index<=j)].mean())
sales['avg_ibex_stock'] = avg_ibex_stock


#### Avg EURUSD Stock Price per Campaign

In [ ]:
## calculating average EURUSD price fot the period of the campaign

avg_eurusd = []
eurusd = yfin.download('EURUSD=X', start=start, end=end)['Adj Close']
for i, j in zip(sales['FECHA_PRIMERA_VENTA'], sales['end_date']):
    avg_eurusd.append(eurusd[(eurusd.index>=i)&(eurusd.index<=j)].mean())
sales['avg_eurusd'] = avg_eurusd


#### Avg US VIX per Campaign

In [ ]:
## calculating average US VIX price fot the period of the campaign

avg_vix = []
vix = yfin.download('^VIX', start=start, end=end)['Adj Close']
for i, j in zip(sales['FECHA_PRIMERA_VENTA'], sales['end_date']):
    avg_vix.append(vix[(vix.index>=i)&(vix.index<=j)].mean())
sales['avg_vix'] = avg_eurusd


In [ ]:
## filling missing values with the mean of their respective column
sales['avg_vix'] = sales['avg_vix'].fillna(sales['avg_vix'].mean())
sales['avg_eurusd'] = sales['avg_eurusd'].fillna(sales['avg_eurusd'].mean())
sales['avg_ibex_stock'] = sales['avg_ibex_stock'].fillna(sales['avg_ibex_stock'].mean())
sales['avg_stock_price'] = sales['avg_stock_price'].fillna(sales['avg_stock_price'].mean())



## FACTURADO related feature

In [ ]:
## creating feature that calculates the average of facturado per cod_tipo from previous year
a = pd.DataFrame(sales.groupby(['MERCADO', 'COD_TIPO', 'year'])[['year','FACTURADO']].mean())
a['mean_l1_facturado_year']= a['FACTURADO'].shift(1)
a.drop('FACTURADO', axis=1,inplace=True)
a = a.where(a['year']!=2018.0).drop('year', axis=1)
sales = sales.merge(a, how = 'left', on = ['MERCADO', 'COD_TIPO','year'])
sales['mean_l1_facturado_year']=sales['mean_l1_facturado_year'].fillna(sales['mean_l1_facturado_year'].mean())    

In [ ]:
sales.info()

In [ ]:
#sales.to_csv('preprocessed_data.csv')